# Final Project

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers
import matplotlib.pyplot as plt

## QB Model

#### Import and Prepare Data

In [2]:
fp = "QB data.xlsx"

In [3]:
def prep(file_path):
    
    df_list = []
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name, nrows=25)
        df['Year'] = sheet_name
        df_list.append(df)
    new = pd.concat(df_list, ignore_index=True)
    # print(new)
    return new
    

In [4]:
all = prep(fp)
all['Awards'] = all['Awards'].apply(lambda x: x[2] if isinstance(x, str) and len(x) >= 3 else "")
all['Year'] = all['Year'].astype(int)

In [5]:
Train = all[all['Year'].apply(lambda x: str(x)[-1] not in ['0', '5'])]
Test = all[all['Year'].apply(lambda x: str(x)[-1] in ['0', '5'])]

X_train = Train.drop(columns=['Rk', 'Player', 'Team', 'Conf', 'Awards'], axis=1)
Y_train = Train['Rk']

X_test_name = Test['Player']
X_test_year = Test['Year']
X_test = Test.drop(columns=['Rk', 'Player', 'Team', 'Conf', 'Awards'], axis=1)
Y_test = Test['Rk']

In [6]:
print(X_test_name)

75           Mac Jones*
76          Kyle Trask*
77         Zach Wilson*
78          Sam Howell*
79      Dillon Gabriel*
             ...       
720          Mike Fouts
721         Patrick Nix
722    Mark Butterfield
723           Mike Groh
724       Tony Graziani
Name: Player, Length: 150, dtype: object


In [7]:
print(X_train, Y_train)

      G  Cmp  Att  Cmp%   Yds  TD   TD%  Int  Int%   Y/A   AY/A   Y/C    Y/G  \
0    15  363  555  65.4  4903  36   6.5   11   2.0   8.8   9.24  13.5  326.9   
1    14  364  470  77.4  4508  45   9.6    3   0.6   9.6  11.22  12.4  322.0   
2    14  302  417  72.4  3941  24   5.8    6   1.4   9.5   9.95  13.0  281.5   
3    13  318  476  66.8  3883  32   6.7    9   1.9   8.2   8.65  12.2  298.7   
4    12  236  327  72.2  3812  40  12.2    4   1.2  11.7  13.55  16.2  317.7   
..   ..  ...  ...   ...   ...  ..   ...  ...   ...   ...    ...   ...    ...   
770  11  196  356  55.1  2608  21   5.9   16   4.5   7.3   6.48  13.3  237.1   
771  11  213  388  54.9  2563  12   3.1   14   3.6   6.6   5.60  12.0  233.0   
772  11  171  280  61.1  2547  15   5.4   11   3.9   9.1   8.40  14.9  231.5   
773  11  179  373  48.0  2490  17   4.6   26   7.0   6.7   4.45  13.9  226.4   
774  11  177  304  58.2  2490  17   5.6    8   2.6   8.2   8.13  14.1  226.4   

      Rate  Year  
0    157.1  2023  
1

In [8]:
#convert to numpy
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

#### Build and Train model

In [9]:
input_weights = tf.constant([1.0, 1.0, 1.0, 1.0, 1.0, 2.0,1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [10]:
model = tf.keras.Sequential([

    # tf.keras.WeightedInputLayer(weights=input_weights, input_shape=(15,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_data=(X_test, Y_test))

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0213 - loss: 50956.2500 - val_accuracy: 0.0000e+00 - val_loss: 7001.5723
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0151 - loss: 4182.3901 - val_accuracy: 0.0400 - val_loss: 259.2960
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0246 - loss: 515.2787 - val_accuracy: 0.0400 - val_loss: 178.4823
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0361 - loss: 194.6718 - val_accuracy: 0.0400 - val_loss: 130.9149
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0304 - loss: 127.8977 - val_accuracy: 0.0400 - val_loss: 112.0829
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0325 - loss: 105.2941 - val_accuracy: 0.0400 - val_loss: 84.8033
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0438 - loss: 84.4752 - val_accuracy: 0.0400 - val_loss: 76.7350
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0432 - loss: 70.5626

#### Test Model

In [11]:
predictions = model.predict(X_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


In [12]:
new = pd.DataFrame()

new.reset_index(drop=True, inplace=True)
X_test_year.reset_index(drop=True, inplace=True)
X_test_name.reset_index(drop=True, inplace=True)

asa = pd.DataFrame(predictions)

new['Prediction'] = asa
new['Name'] = X_test_name
new['Year'] = X_test_year

grouped = new.groupby('Year')
years = []

for year, group in grouped:
    globals()[f"df_{year}"] = group
    years.append(f"df_{year}")
    

In [13]:
for i in years:
    globals()[f"{i}"] = globals()[f"{i}"].sort_values(by='Prediction', ascending=True).head(3)

top = Test
top = top.iloc[0:0]
top.to_csv("Top_QB.csv", index=False)

for i in years:
    for name in globals()[f"{i}"]['Name']:
        # print(name)
        r = Test[Test.apply(lambda row: row.astype(str).str.contains(name).any(), axis=1)] 
        r.reset_index(drop=True, inplace=True)        
        r.to_csv("Top_QB.csv", mode='a', index=False, header=False)

#### Output top 3 QB for each year tested

In [14]:
df = pd.read_csv("Top_QB.csv")
print(df)

    Rk            Player              Team      Conf   G  Cmp  Att  Cmp%  \
0    2    Cody Ledbetter  New Mexico State  Big West  11  259  453  57.2   
1   14   Marcus Crandell     East Carolina       Ind  11  235  447  52.6   
2    6     Charlie Batch  Eastern Michigan       MAC  11  244  421  58.0   
3    2    Jared Lorenzen          Kentucky       SEC  11  321  559  57.4   
4    3   Kliff Kingsbury        Texas Tech    Big 12  12  362  585  61.9   
5    1      Chris Weinke     Florida State       ACC  12  266  431  61.7   
6    8       Luke Getsy*             Akron       MAC  13  278  525  53.0   
7    2      Cody Hodges*        Texas Tech    Big 12  12  353  531  66.5   
8    1      Colt Brennan            Hawaii       WAC  12  350  515  68.0   
9    2     Landry Jones*          Oklahoma    Big 12  14  405  617  65.6   
10   1     Bryant Moniz*            Hawaii       WAC  14  361  555  65.0   
11   4  Dominique Davis*     East Carolina      CUSA  13  393  609  64.5   
12   2     M

## RB Model

#where the csv files are stored

In [15]:
folder_path = 'RB_DATA'

#all csv files for RBs

In [16]:
datasets = ['RB_DATA_1993.csv', 'RB_DATA_1994.csv', 'RB_DATA_1995.csv', 'RB_DATA_1996.csv', 'RB_DATA_1997.csv', 'RB_DATA_1998.csv', 'RB_DATA_1999.csv', 'RB_DATA_2000.csv', 'RB_DATA_2002.csv', 'RB_DATA_2003.csv', 'RB_DATA_2004.csv', 'RB_DATA_2005.csv', 'RB_DATA_2006.csv', 'RB_DATA_2007.csv', 'RB_DATA_2008.csv', 'RB_DATA_2009.csv','RB_DATA_2010.csv', 'RB_DATA_2011.csv', 'RB_DATA_2012.csv', 'RB_DATA_2013.csv', 'RB_DATA_2014.csv', 'RB_DATA_2015.csv','RB_DATA_2016.csv', 'RB_DATA_2017.csv', 'RB_DATA_2018.csv', 'RB_DATA_2019.csv', 'RB_DATA_2020.csv', 'RB_DATA_2021.csv', 'RB_DATA_2022.csv', 'RB_DATA_2023.csv' ]  # List to store datasets

#list to store player names

In [17]:
player_names_all = []

#Lists to store training and testing results

In [18]:
train_results = []
test_results = []

#Create an empty set to store all unique teams and conferences across datasets

In [19]:
all_teams = set()
all_confs = set()

#collecting a full set of unique teams and conferences to use them for one-hot encoding

In [20]:
# First pass through datasets to collect all unique teams and conferences
for dataset in datasets:
    file_path = os.path.join(folder_path, dataset)
    df = pd.read_csv(file_path)
    
    # Collect all unique teams and conferences
    all_teams.update(df['Team'].unique())
    all_confs.update(df['Conf'].unique())

all_teams = sorted(list(all_teams))
all_confs = sorted(list(all_confs))

#Loop through datasets to process them, lots of code in the loop so it is commented

In [21]:
for dataset in datasets:
    # Extract the year from the dataset name
    year = dataset.split('_')[-1].split('.')[0]
    file_path = os.path.join(folder_path, dataset)
    df = pd.read_csv(file_path)
    
    # Ensure 'Awards' column exists and clean it
    if 'Awards' not in df.columns:
        print(f"Warning: 'Awards' column not found in {dataset}. Skipping this dataset.")
        continue
    
    df['Awards'] = df['Awards'].fillna(0).astype(int)  # Clean 'Awards' column

    # Check for Heisman winner (only one should be present)
    heisman_winners = df[df['Awards'] == 1]
    if len(heisman_winners) != 1:
        print(f"Warning: More than one or no Heisman winner found in {year} dataset!")
        continue  # Skip this dataset if there's no valid Heisman winner
    
    # Track player names
    player_names_all.append(df['Player'].values)
    
    # Prepare features and target
    X = df.drop(['Awards', 'Player'], axis=1).select_dtypes(include=['number'])
    y = df['Awards']

    # One-Hot Encoding for 'Team' and 'Conf', ensuring consistency across datasets
    X_encoded = pd.concat([
        X,
        pd.get_dummies(df['Team'], prefix='Team').reindex(columns=all_teams, fill_value=0),
        pd.get_dummies(df['Conf'], prefix='Conf').reindex(columns=all_confs, fill_value=0)
    ], axis=1)

    # Reset index for consistency
    df = df.reset_index(drop=True)
    
    # Check if this year should be used for testing or training
    if year[-1] in ['0', '5']:  # Testing set for years ending in '0' or '5'
        # Split data into test set
        X_test = X_encoded
        y_test = y

        # Scale the data for testing
        scaler = StandardScaler()
        X_test_scaled = scaler.fit_transform(X_test)
        
        # Build the model
        model = tf.keras.Sequential([ 
            tf.keras.layers.Dense(128, activation='relu', input_shape=(X_test_scaled.shape[1],)),  # Increased complexity
            tf.keras.layers.Dropout(0.2),  # Add dropout to prevent overfitting
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dropout(0.2),  # Add dropout here as well
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid') 
        ])

        # Compile the model with adjusted learning rate and class weight
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Adjust learning rate
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
        
        # Train the model on the training set from previous years
        if len(train_results) > 0:
            X_train_scaled = np.concatenate([result['X_train_scaled'] for result in train_results], axis=0)
            y_train = np.concatenate([result['y_train'] for result in train_results], axis=0)
            
            model.fit(X_train_scaled, y_train, epochs=30, batch_size=32, verbose=0)

            # Predict on the test set and get probabilities
            y_pred_prob = model.predict(X_test_scaled)

            # Debugging: Print out the predicted probabilities to check for improvement
            print(f"Predicted probabilities for year {year}:")
            print(y_pred_prob[:10])  # Display first 10 predictions
                    
        # Retrieve player names for the test set
        test_player_names = df.loc[X_test.index, 'Player'].values

        # Prepare the test data for comparison
        test_data = X_test.copy()
        test_data['Actual Heisman'] = y_test.values
        test_data['Player'] = test_player_names
        test_data['Predicted Probability'] = y_pred_prob

        print(f"Data for year {year}:")
        print(test_data[['Player', 'Actual Heisman', 'Predicted Probability']].head()) 

        # Sort by predicted probability to get the top 3 predicted winners
        top_3_predicted = test_data.nlargest(3, 'Predicted Probability')[['Player', 'Predicted Probability']]

        # Get player statistics for the top 3 by looking them up in the original dataset
        top_3_stats = df[df['Player'].isin(top_3_predicted['Player'].values)][[
            'Player', 'Team', 'Conf', 'G', 'Att', 'Yds', 'Y/A', 'TD', 'Y/G', 'Rec', 'Yds.1', 
            'Y/R', 'TD.1', 'Y/G.1', 'Plays', 'Yds.2', 'Avg', 'TD.2'
        ]]

        # Add a custom identifier with format 'player-name-id'
        top_3_stats['-9999'] = top_3_stats['Player'].str.lower().str.replace(' ', '-')
        
        # Save the top 3 players and their stats for this year in the same format as the original CSV
        output_file_path = os.path.join('RB_DATA', f'top_3_players_{year}.csv')
        top_3_stats.to_csv(output_file_path, index=False)

        # Check if there is at least one actual Heisman winner
        actual_winner = test_data[test_data['Actual Heisman'] == 1]
        
        if actual_winner.empty:
            print(f"Warning: No actual Heisman winner found for {year}. Skipping this dataset.")
            continue  # Skip this year if there's no actual winner

        # If there's an actual winner, get the name
        actual_winner_name = actual_winner['Player'].values[0]
        print(f"Actual Heisman Winner for {year}: {actual_winner_name}")  

        # Check if one of the predicted winners matches the actual winner
        predicted_winners = top_3_predicted['Player'].values
        match = 'Yes' if actual_winner_name in predicted_winners else 'No'

        # Store the test results for this year
        test_results.append({
            'Year': year,
            'Predicted Winners': ', '.join(predicted_winners),
            'Actual Winner': actual_winner_name,
            'Match': match
        })

    else:  # Training set for all other years
        # Use the entire dataset as the training set
        X_train = X_encoded
        y_train = y
        
        # Scale the data for training
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)

        # Store training results for later use
        train_results.append({
            'X_train_scaled': X_train_scaled,
            'y_train': y_train
        })
        
test_results_df = pd.DataFrame(test_results)

#printing top 3 from each test year
print(test_results_df)

C:\Users\fwdav\PycharmProjects\ece5831-2024-assignments\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted probabilities for year 1995:
[[0.30075973]
 [0.30075973]
 [0.30075973]
 [0.30075973]
 [0.30075973]
 [0.30075973]
 [0.30075973]
 [0.30075973]
 [0.30075973]
 [0.30075973]]
Data for year 1995:
                  Player  Actual Heisman  Predicted Probability
0             Troy Davis               0                0.30076
1            Wasean Tait               0                0.30076
2           George Jones               0                0.30076
3           Eddie George               1                0.30076
4  Tshimanga Biakabutuka               0                0.30076
Actual Heisman Winner for 1995: Eddie George


C:\Users\fwdav\PycharmProjects\ece5831-2024-assignments\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted probabilities for year 2000:
[[0.05083998]
 [0.05083998]
 [0.05083998]
 [0.05083998]
 [0.05083998]
 [0.05083998]
 [0.05083998]
 [0.05083998]
 [0.05083998]
 [0.05083998]]
Data for year 2000:
                Player  Actual Heisman  Predicted Probability
0  LaDainian Tomlinson               1                0.05084
1      Damien Anderson               0                0.05084
2      Michael Bennett               0                0.05084
3      Deonce Whitaker               0                0.05084
4       Robert Sanford               0                0.05084
Actual Heisman Winner for 2000: LaDainian Tomlinson


C:\Users\fwdav\PycharmProjects\ece5831-2024-assignments\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted probabilities for year 2010:
[[0.05147699]
 [0.05147699]
 [0.05147699]
 [0.05147699]
 [0.05147699]
 [0.05147699]
 [0.05147699]
 [0.05147699]
 [0.05147699]
 [0.05147699]]
Data for year 2010:
             Player  Actual Heisman  Predicted Probability
0  LaMichael James*               1               0.051477
1  Denard Robinson*               0               0.051477
2   Mikel Leshoure*               0               0.051477
3    Jordan Todman*               0               0.051477
4      Bobby Rainey               0               0.051477
Actual Heisman Winner for 2010: LaMichael James*


C:\Users\fwdav\PycharmProjects\ece5831-2024-assignments\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted probabilities for year 2015:
[[0.0506715]
 [0.0506715]
 [0.0506715]
 [0.0506715]
 [0.0506715]
 [0.0506715]
 [0.0506715]
 [0.0506715]
 [0.0506715]
 [0.0506715]]
Data for year 2015:
                 Player  Actual Heisman  Predicted Probability
0        Derrick Henry*               1               0.050672
1  Christian McCaffrey*               0               0.050672
2    Leonard Fournette*               0               0.050672
3        Royce Freeman*               0               0.050672
4      Ezekiel Elliott*               0               0.050672
Actual Heisman Winner for 2015: Derrick Henry*


C:\Users\fwdav\PycharmProjects\ece5831-2024-assignments\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted probabilities for year 2020:
[[0.04983077]
 [0.04983077]
 [0.04983077]
 [0.04983077]
 [0.04983077]
 [0.04983077]
 [0.04983077]
 [0.04983077]
 [0.04983077]
 [0.04983077]]
Data for year 2020:
               Player  Actual Heisman  Predicted Probability
0        Breece Hall*               0               0.049831
1  Sincere McCormick*               0               0.049831
2       Najee Harris*               1               0.049831
3     Michael Carter*               0               0.049831
4      Khalil Herbert               0               0.049831
Actual Heisman Winner for 2020: Najee Harris*
   Year                                  Predicted Winners  \
0  1995              Troy Davis, Wasean Tait, George Jones   
1  2000  Sultan McCullough, Michael Wallace, LaDainian ...   
2  2005  DeAngelo Williams*, Jerome Harrison, Reggie Bush*   
3  2010  LaMichael James*, Denard Robinson*, Mikel Lesh...   
4  2015  Derrick Henry*, Christian McCaf

## QB vs RB Model
#### Import and Preprocess Data

In [22]:
input_df = pd.read_csv('Heisman_Winner_QB_vs_RB_Singles.csv')
#print(input_data_cv.head())

#Standardize stats by # of games played
input_df['Pass_Cmp_per_Game_1'] = input_df['Pass_Cmp_1']/input_df['QB_Games_1']
input_df['Pass_Att_per_Game_1'] = input_df['Pass_Att_1']/input_df['QB_Games_1']
input_df['Pass_TD_per_Game_1'] = input_df['Pass_TD_1']/input_df['QB_Games_1']
input_df['Int_per_Game_1'] = input_df['Int_1']/input_df['QB_Games_1']

input_df['Rush_Att_per_Game_1'] = input_df['Rush_Att_1']/input_df['RB_Games_1']
input_df['Rush_Yds_per_Game_1'] = input_df['Rush_Yds_1']/input_df['RB_Games_1']
input_df['Rush_TD_per_Game_1'] = input_df['Rush_TD_1']/input_df['RB_Games_1']
input_df['Rec_per_Game_1'] = input_df['Rec_1']/input_df['RB_Games_1']
input_df['Rcv_Yds_per_Game_1'] = input_df['Rcv_Yds_1']/input_df['RB_Games_1']
input_df['Rcv_TD_per_Game_1'] = input_df['Rcv_TD_1']/input_df['RB_Games_1']
input_df['ScrmgPlays_per_Game_1'] = input_df['ScrmgPlays_1']/input_df['RB_Games_1']
input_df['Scrmg_Yds_per_Game_1'] = input_df['Scrmg_Yds_1']/input_df['RB_Games_1']
input_df['Scrmg_TD_per_Game_1'] = input_df['Scrmg_TD_1']/input_df['RB_Games_1']


print(input_df.head())

   Year             QB_1 QB_Conf_1  QB_Rank_1  QB_Games_1  Pass_Cmp_1  \
0  2023   Jayden Daniels       SEC          5          12         236   
1  2022   Caleb Williams    Pac-12          3          14         333   
2  2021      Bryce Young       SEC          2          15         366   
3  2020  Trevor Lawrence       ACC          8          10         231   
4  2019       Joe Burrow       SEC          1          15         402   

   Pass_Att_1  Cmp%_1  Pass_Yds_1  Pass_TD_1  ...  Int_per_Game_1  \
0         327    72.2        3812         40  ...        0.333333   
1         500    66.6        4537         42  ...        0.357143   
2         547    66.9        4872         47  ...        0.466667   
3         334    69.2        3153         24  ...        0.500000   
4         527    76.3        5671         60  ...        0.400000   

   Rush_Att_per_Game_1  Rush_Yds_per_Game_1  Rush_TD_per_Game_1  \
0            20.357143           123.714286                 1.5   
1           

In [23]:
#Create new column to indicate which player won the Heisman
input_df['Winner_Coded'] = '999'
for row in range(input_df.shape[0]):
    if input_df['Winner'][row] == input_df['QB_1'][row]:
        input_df.loc[row, 'Winner_Coded'] = 0.0
    if input_df['Winner'][row] == input_df['RB_1'][row]:
        input_df.loc[row, 'Winner_Coded'] = 1.0
print(input_df['Winner_Coded'])

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     1.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    1.0
15    0.0
16    0.0
17    0.0
18    1.0
19    0.0
20    0.0
21    0.0
22    0.0
23    1.0
24    1.0
25    0.0
26    0.0
27    1.0
28    1.0
29    0.0
Name: Winner_Coded, dtype: object


In [24]:
#Format conference fields 
print(input_df['QB_Conf_1'].unique())
print(input_df['RB_Conf_1'].unique())
input_df['QB_Conf_1'] = input_df['QB_Conf_1'].str.strip()
input_df['RB_Conf_1'] = input_df['RB_Conf_1'].str.strip()

input_df['RB_Conf_1'] = input_df['RB_Conf_1'].replace({'Pac-10':'Pac-12'})
print(input_df['QB_Conf_1'].unique())
print(input_df['RB_Conf_1'].unique())

#Create dummy vars for  conferences
pd.concat([input_df, pd.get_dummies(input_df['QB_Conf_1'],prefix='QB1_').astype(int)], axis=1)
pd.concat([input_df, pd.get_dummies(input_df['RB_Conf_1'],prefix='RB1_').astype(int)], axis=1)

['SEC' 'Pac-12' 'ACC' 'Big 12' 'Big Ten']
['Big 12' 'Big Ten' 'SEC' 'ACC' 'Pac-12' 'WAC']
['SEC' 'Pac-12' 'ACC' 'Big 12' 'Big Ten']
['Big 12' 'Big Ten' 'SEC' 'ACC' 'Pac-12' 'WAC']


,Year,QB_1,QB_Conf_1,QB_Rank_1,QB_Games_1,Pass_Cmp_1,Pass_Att_1,Cmp%_1,Pass_Yds_1,Pass_TD_1,...,ScrmgPlays_per_Game_1,Scrmg_Yds_per_Game_1,Scrmg_TD_per_Game_1,Winner_Coded,RB1__ACC,RB1__Big 12,RB1__Big Ten,RB1__Pac-12,RB1__SEC,RB1__WAC
0,2023,Jayden Daniels,SEC,5,12,236,327,72.2,3812,40,...,23.142857,147.285714,1.571429,0.0,0,1,0,0,0,0
1,2022,Caleb Williams,Pac-12,3,14,333,500,66.6,4537,42,...,21.500000,128.583333,1.583333,0.0,0,0,1,0,0,0
2,2021,Bryce Young,SEC,2,15,366,547,66.9,4872,47,...,23.000000,143.750000,1.583333,0.0,0,0,1,0,0,0
3,2020,Trevor Lawrence,ACC,8,10,231,334,69.2,3153,24,...,22.615385,145.461538,2.307692,0.0,0,0,0,0,1,0
4,2019,Joe Burrow,SEC,1,15,402,527,76.3,5671,60,...,24.714286,161.071429,1.857143,0.0,0,0,1,0,0,0
5,2018,Kyler Murray,Big 12,3,14,260,377,69.0,4361,42,...,14.400000,115.733333,1.733333,0.0,1,0,0,0,0,0
6,2017,Baker Mayfield,Big 12,2,14,285,404,70.5,4627,43,...,20.692308,165.461538,1.461538,0.0,0,0,0,1,0,0
7,2016,Lamar Jackson,ACC,16,13,230,409,56.2,3543,30,...,30.000000,191.181818,1.363636,0.0,0,1,0,0,0,0
8,2015,Deshaun Watson,ACC,9,15,333,491,67.8,4104,35,...,27.066667,154.000000,1.866667,1.0,0,0,0,0,1,0
9,2014,Marcus Mariota,Pac-12,3,15,304,445,68.3,4454,42,...,25.857143,195.714286,2.285714,0.0,0,0,1,0,0,0


In [25]:
#Drop non-standardized variables
vars_final = input_df.drop(columns = ['Pass_Cmp_1','Pass_Att_1','Pass_Yds_1', 'Pass_TD_1', 'Int_1', 'Rush_Att_1',
                                      'Rush_Yds_1', 'Rush_TD_1', 'Rec_1','Rcv_Yds_1','Rcv_TD_1','ScrmgPlays_1',
                                      'Scrmg_Yds_1','Scrmg_TD_1',
                                      'Winner', 'QB_Conf_1', 'RB_Conf_1'])

#Drop variables we won't be using to predict
vars_final.drop(columns = ['QB_1', 'RB_1'], inplace=True)
#vars_final.drop(columns = ['QB_Rank_1', 'QB_Rank_2', 'QB_Rank_3', 'RB_Rank_1', 'RB_Rank_2', 'RB_Rank_3',
                           #'QB_Games_1', 'QB_Games_2', 'QB_Games_3', 'RB_Games_1', 'RB_Games_2', 'RB_Games_3'], inplace=True)

In [26]:
print(vars_final.columns)

Index(['Year', 'QB_Rank_1', 'QB_Games_1', 'Cmp%_1', 'Pass_TD%_1', 'Int%_1',
       'Y_per_Atmpt_1', 'Adj_Y_per_A_1', 'Y_per_C_1', 'Y_per_G_1',
       'Pass_Effncy_Rate_1', 'RB_Rank_1', 'RB_Games_1', 'Rush_Avg_1',
       'Rcv_Avg_1', 'Scrmg_Avg_1', 'Pass_Cmp_per_Game_1',
       'Pass_Att_per_Game_1', 'Pass_TD_per_Game_1', 'Int_per_Game_1',
       'Rush_Att_per_Game_1', 'Rush_Yds_per_Game_1', 'Rush_TD_per_Game_1',
       'Rec_per_Game_1', 'Rcv_Yds_per_Game_1', 'Rcv_TD_per_Game_1',
       'ScrmgPlays_per_Game_1', 'Scrmg_Yds_per_Game_1', 'Scrmg_TD_per_Game_1',
       'Winner_Coded'],
      dtype='object')


In [27]:
#Create X and y tensors
X = vars_final.drop(columns = ['Year','Winner_Coded'])
y = vars_final['Winner_Coded']

In [28]:
#Standardize X
train_mean = X.mean(axis=0)
X -= train_mean
train_std = X.std(axis=0)
X /= train_std

In [29]:
print(X)

    QB_Rank_1  QB_Games_1    Cmp%_1  Pass_TD%_1    Int%_1  Y_per_Atmpt_1  \
0   -0.475936   -0.732828  1.260621    2.001964 -1.138228       1.956380   
1   -0.602853    0.732828  0.142436   -0.253213 -1.414721      -0.284408   
2   -0.666311    1.465656  0.202338   -0.134519 -0.999982      -0.456776   
3   -0.285562   -2.198484  0.661593   -0.965374 -0.723489      -0.025855   
4   -0.729769    1.465656  2.079293    1.527190 -1.276475       1.180723   
5   -0.602853    0.732828  0.621658    1.349150 -0.170504       1.870196   
6   -0.666311    0.732828  0.921172    1.052416 -0.723489       1.784012   
7    0.222104    0.000000 -1.934195   -0.906027  0.244235      -0.629144   
8   -0.222104    1.465656  0.382047   -1.024721  0.797221      -0.887697   
9   -0.602853    1.465656  0.481885    0.340255 -1.552968       0.491250   
10  -0.222104    0.732828  0.202338    0.933723  0.797221       1.008355   
11   0.158645    0.000000  0.421982   -1.677535  0.105989      -0.801513   
12  -0.41247

In [30]:

#Create model
def build_network(X):
    #normal_layer = layers.Normalization(axis=-1)
    #normal_layer.adapt(np.array(X.values))
    #model.add(normal_layer)
    model = tf.keras.Sequential([
        layers.Dense(10, activation = 'relu'),
        layers.Dense(5, activation = 'relu'),
        layers.Dense(1, activation = 'sigmoid')
    ])

    model.compile(loss = 'binary_crossentropy',
                           optimizer = tf.keras.optimizers.Adam(learning_rate = 0.1),
                           metrics = ['accuracy'])

    return model

In [31]:
def leave_one_out_train(X, y):
    callbacks =  [
    tf.keras.callbacks.EarlyStopping(
        monitor = 'loss', 
        patience=3
    ),
    tf.keras.callbacks.TensorBoard()
    ]

    losses = []
    accuracies = []
    for i in range(len(y)):
        #print(y.shape)
        y_train = np.asarray(y)
        y_test = np.asarray(y[i])
        y_train = np.delete(y_train, i)
        X_train = X.copy()
        X_test = pd.DataFrame(X.iloc[i,:]).transpose()
        X_train.drop([i], inplace=True)
        y_train = tf.convert_to_tensor(y_train, dtype='float')
        y_test = tf.convert_to_tensor(y_test, dtype='float')
        y_test = tf.reshape(y_test, [1,1])

        my_model = build_network(X)
        my_model.fit(X_train, y_train, epochs = 10, batch_size = 4, callbacks = callbacks)
        error = my_model.evaluate(X_test, y_test)
        losses.append(error[0])
        accuracies.append(error[1])
        print(f'Finished round {i+1}')
    return losses, accuracies


In [32]:
loss_list, accuracy_list = leave_one_out_train(X, y)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4757 - loss: 0.6956      
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6987 - loss: 0.6105 
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7047 - loss: 0.6251 
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6806 - loss: 0.6678 
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7690 - loss: 0.5467 
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8075 - loss: 0.4968 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 1.0000 - loss: 0.2475
Finished round 1
Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6504 - loss: 0.9158  
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7644 - loss: 0.6132 
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7156 - loss: 0.5913 
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7172 - loss: 0.6096 
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6690 - loss:

In [33]:
print(f'Average Loss: {np.average(loss_list)}')
print(f'Average Accuracy: {np.average(accuracy_list)}')

Average Loss: 0.5865769346555074
Average Accuracy: 0.7666666666666667


In [34]:
callbacks =  [
    tf.keras.callbacks.EarlyStopping(
        monitor = 'loss', 
        patience=3
    ),
    tf.keras.callbacks.TensorBoard()
    ]
final_model = build_network(X)
y = tf.convert_to_tensor(y, dtype='float')
final_model.fit(X, y, epochs = 10, batch_size = 8, callbacks = callbacks)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6550 - loss: 0.9900
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8025 - loss: 0.6187 
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7567 - loss: 0.5812 
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7692 - loss: 0.5475 
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7817 - loss: 0.5298 
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8358 - loss: 0.4550 
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7567 - loss: 0.5688 
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8233 - loss: 0.4707 
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8025 - loss: 0.5018 
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7692 - loss: 0.5450 


#### Python Package Import (same functionality as above)

In [35]:
from qb_vs_rb import QBvsRB

In [36]:
new_qbrb = QBvsRB()

In [37]:
new_qbrb.load_train_data('Heisman_Winner_QB_vs_RB_Singles.csv')

In [38]:
new_qbrb.make_x_y()

In [39]:
new_qbrb.leave_one_out_cross_val()

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4401 - loss: 0.9041   
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7565 - loss: 0.5934 
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7288 - loss: 0.5816 
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7079 - loss: 0.6217 
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7364 - loss: 0.5960 
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7292 - loss: 0.5952 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 1.0000 - loss: 0.3166
Finished round 1
Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2910 - loss: 0.8212
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7357 - loss: 0.5877 
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8022 - loss: 0.5169 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 1.0000 - loss: 0.2032
Finished round 2
Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6521 - los

In [40]:
new_qbrb.train_final_model()

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3458 - loss: 0.8261   
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7900 - loss: 0.5410 
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8150 - loss: 0.4895 
